# 十徳ナイフとしてのGradle

[JJUG CCC 2016 Spring](http://www.java-users.jp/?page_id=2377) / [M-6_1 十徳ナイフとしてのGradle](http://www.java-users.jp/?page_id=2396#M-6_1)

*[#ccc_m61](https://twitter.com/search?f=tweets&q=%23ccc_m61&src=typd)*

## 自己紹介

よしだ

* twitter: [@grimrose](https://twitter.com/grimrose) /とーます 
* github: [grimrose](https://github.com/grimrose)
* Community: [Yokohama.groovy](http://connpass.com/series/253/)
![icon](https://avatars3.githubusercontent.com/u/623724?v=3&s=280)


好きな言語: ![Groovy Logo](https://upload.wikimedia.org/wikipedia/commons/3/36/Groovy-logo.svg)


好きなIDE: ![IntelliJ IDEA](img/icon_IntelliJIDEA.png)


## お仕事

* 所属: とある人材紹介会社のマーケティング部門
* 役割: データ分析チームのデータエンジニア


## お仕事

社内BIツールの開発、設計、運用

* [Skinny](http://skinny-framework.org), [ScalikeJDBC](http://scalikejdbc.org), [Scalaz](https://github.com/scalaz/scalaz), [Hazelcast](http://hazelcast.org), [scalaxb](http://scalaxb.org/ja), [Akka](http://akka.io), [Embulk](http://www.embulk.org/)

* [Laravel](https://laravel.com), [FuelPHP](http://fuelphp.com), [Angular](https://angularjs.org), [TypeScript](https://www.typescriptlang.org), [Wijmo5](http://wijmo.c1.grapecity.com/products/wijmo-5/)
* [Electron](http://electron.atom.io), [Mithril.js](http://mithril-ja.js.org)
* etc.

## Agenda

* Gradleとは？
* 今日話す内容と「Gradle 徹底入門」
* タスクランナーとしてのGradle
* Gradleを使った自動化例
* Pluginの紹介


## [Gradle](http://gradle.org/)とは？

* 全く知らない？
* 聞いたことがある？
* 触ったことがある？
* 使っている？
* Plugin書いてる？
* 俺がGradleだ！


* 次世代ビルドツール
* version: 2.13
* 主にJava、Groovyで実装
* Java8対応, Java9も対応開始
* Groovy: 2.4
* [sdkman](http://sdkman.io/index.html)を使うとインストールが簡単

Gradle Wrapperのアップデート

```shell
./gradlew wrapper --gradle-version=2.13
```

## NEW!!

build.gradleが[Kotlin](http://kotlinlang.org/)で書けるように

* [Gradle Meets Kotlin](http://blog.jetbrains.com/kotlin/2016/05/gradle-meets-kotlin/)

* [Kotlin Meets Gradle](https://gradle.org/blog/kotlin-meets-gradle/)

```groovy
apply plugin: 'application'

mainClassName = 'samples.HelloWorld'

repositories {
    jcenter()
}

dependencies {
    testCompile "junit:junit:4.12"
}
```

[gradle/gradle-script-kotlin](https://github.com/gradle/gradle-script-kotlin)

```kotlin
import org.gradle.api.plugins.*
import org.gradle.script.lang.kotlin.*

apply<ApplicationPlugin>()

configure<ApplicationPluginConvention> {
    mainClassName = "samples.HelloWorld"
}

repositories {
    jcenter()
}

dependencies {
    "testCompile"("junit:junit:4.12")
}
```

その他、Gradleについては
[Gradleで始めるGroovy expand](https://speakerdeck.com/grimrose/gradledeshi-merugroovy-expand-number-jjug-ccc-number-ccc-r14)
でも紹介しているので、参考にしてください。


### Gradle徹底入門

![Gradle徹底入門](http://www.seshop.com/static/images/product/17414/L.png)

* 第2部 基礎編 6章 スクリプトファイルの記述
* 第4部 発展編 13章 エキスパートへの道

## build.gradle

ざっくりと言うと以下の内容

* 利用しているプラグイン
* ライブラリの依存関係
* プラグインの設定


その他に

* タスクの定義
* ソースファイルの位置
* メソッド、クラス


つまり、

すっきりしている**build.gradle**ほど

よりGradle Pluginに依存している

[Groovyを知らない人のためのbuild.gradle読み書き入門](http://qiita.com/opengl-8080/items/a0bb31fb20cb6505188b)

## タスクランナーとしてのGradle

Gradleはタスクを組み合わせて、DAG(有向非循環グラフ)を作る 

1. **実行したいことをタスクに落とし込む**
1. **タスクを繋げる**
1. **ワンクリックで実行できるようにする**

### タスクの依存関係

後に実行したいタスク名**.dependsOn** 先に実行するタスク名

* 先に実行するタスクは、配列またはクロージャで渡すことが出来る。
* タスクが循環参照する場合は、例外が発生する。

### タスクの順位付け

後に実行したいタスク名**.mustRunAfter** 先に実行するタスク名

後に実行したいタスク名**.shouldRunAfter** 先に実行するタスク名

* 依存関係よりも緩やかな制約を付ける場合に利用する。
* mustを通常使うとよい。
* shouldは例外ケースに該当したら無視されるので、それでも問題ない時のみがよい。

### タスクの後始末

後始末が必要なタスク名**.finalizedBy** 後始末を行うタスク名

* 一連のタスクで生成した中間ファイルや、起動したプロセスの停止をしたい場合に利用する。
* 前のタスクで例外やエラーになっても実行される。

### タスクの条件実行

条件によって実行したいタスク名**.onlyIf** { 条件式 }

* 条件式の戻り値をbool値にする。
* コマンドの引数や環境変数によって変えたりしたい場合等。


### UP-TO-DATEでタスクをスキップ

タスクの**inputs, outputs**プロパティのfileやdirへ、ファイルパスやディレクトリパスを指定すると、更新に応じてタスクをスキップする事が出来る。

**outputs.upToDateWhen{ true }**とすると常に最新として扱われるので、**inputs**に更新があった場合のみ、タスクが実行されるようになる。


参考:

* Gradle徹底入門 
    * 6.2.2 Taskオブジェクト
    * 6.3.2 依存関係の定義方法
    * 6.3.4 タスクの制御
    * 6.3.7 タスクの順位付け
    * 6.3.8 ファイナライザータスク


* [17.4. Adding dependencies to a task](https://docs.gradle.org/current/userguide/more_about_tasks.html#sec:adding_dependencies_to_tasks)
* [17.5. Ordering tasks](https://docs.gradle.org/current/userguide/more_about_tasks.html#sec:ordering_tasks)
* [17.8. Skipping tasks](https://docs.gradle.org/current/userguide/more_about_tasks.html#N112CB)
* [17.9. Skipping tasks that are up-to-date](https://docs.gradle.org/current/userguide/more_about_tasks.html#sec:up_to_date_checks)
* [17.11. Finalizer tasks](https://docs.gradle.org/current/userguide/more_about_tasks.html#N113D6)

### build.gradleの分割

タスクが多くなってbuild.gradleが見づらくなってきた。

-> build.gradleを分割してみましょう。


build.gradleを以下のように記載。

```groovy
apply from: 'gradle/front.gradle'
// URLも利用可能
apply from: 'http://example.com/extra.gradle'

// releaseのみ含めたい場合
if (ext.env == 'release') {
  apply from: 'gradle/release.gradle'
}
```


### 注意!!

Gradle Pluginを使ったタスクや

Pluginのクラスを利用する場合は、

classpathが読み込まれるか確認しておく

## Gradleを使った自動化のタスク例

* [Apache POI](https://poi.apache.org)を使ってxlsxからcsvへ
* [H2](http://www.h2database.com/html/main.html)を使ってcsvからSQLへ
* windowsからLinuxのサーバへファイルを条件に応じて転送する
* リモートに置いてあるファイルとの差分をチェックする
* ブラウザから指定のリンクが活きてるかをチェックする
* ローカルから[Jenkins](https://jenkins.io)のjobをRESTで実行する
* etc.


### 自動化するタスク例

低頻度で発生する作業だけど、毎回手でやるには面倒くさい。

でも、システム化するには工数がかかる。

既にExcelで運用管理しているが、改善の優先順位が低い。

そういったタスクをいかに楽して自動化するか？

* PowerShell?
* ShellScript(cygwin)?
* JScript ?
* .bat or .cmd ?


もしかしてVBですかーッ？

**またまた、ご冗談を(AA略**

## なぜGradleを使うのか

* Pluginがあれば、タスクの組み合わせで出来る。
* Pluginが無ければ、一旦タスクにしてみる。
* buildscriptブロックにライブラリを書けば、タスクで利用できる。
* Gradle TestKitでを使うことで、テストが[Spock](https://github.com/spockframework/spock)で書ける。
* Gradleに依存する部分としない部分を切り分けておけば、再利用しやすい。


### なぜGradleを使うのか

* ファイルの移動、コピー、削除程度なら[CopySpec](https://docs.gradle.org/current/javadoc/org/gradle/api/file/CopySpec.html)の基本をおさえておけば十分活用できる。
* ライブラリを呼ばなくてもJSON, JDBCラッパー等、Gradleが用意しているGroovyの標準機能で十分まかなえる。
* 大きくなったら、クラスにして別のプロジェクトとして切り出すことも可能。

オレオレTaskクラスは、Groovyだけでなく、もちろんJavaや他のJVM言語でも書けます！

* 参考: [Chapter 38. Writing Custom Task Classes](https://docs.gradle.org/current/userguide/custom_tasks.html)

タスクをクラスにして別のプロジェクトとして切り出す

-> **Gradle Plugin** 

## Gradle Pluginの紹介

### [Gradle - Plugins](https://plugins.gradle.org/)

### Gradle Pluginを知るポイント

* READMEを読む


### Gradle Pluginを知るポイント

* `src/main/resources/META-INF/gradle-plugins/`のpropertiesファイルにあるPluginのクラスを見つける
* Pluginクラスの`apply`メソッドを探す
    * タスクの定義やExtensionの設定をしている為
* Extensionクラスを探す
    * Pluginが利用する設定の名前や型を知る為

### Groovyに関するPlugin

REPL(groovysh)を使いたい

* [com.github.tkruse.groovysh](https://plugins.gradle.org/plugin/com.github.tkruse.groovysh)

groovy scriptをgradleから実行したい

* [com.mrhaki.groovyrun](https://plugins.gradle.org/plugin/com.mrhaki.groovyrun)
    * **SimpleHttpServer**が付いているので、簡易HTTPサーバが必要なときもこれで。

### Download Task Plugin

外部リソースをダウンロードしたい

* [de.undercouch.download](https://plugins.gradle.org/plugin/de.undercouch.download)


### SSH Plugin

[org.hidetake.ssh](https://plugins.gradle.org/plugin/org.hidetake.ssh)

* ファイルを転送する
* リモートのTomcat, Apacheの再起動等
* SSHクライアントの代替


### フロントエンドとの連携に関するPlugin

CoffeeScriptやLESSなどのビルドをまとめて実行したい

* [com.bertramlabs.asset-pipeline](https://plugins.gradle.org/plugin/com.bertramlabs.asset-pipeline)

フロントエンドのNode.jsのタスクを実行したい

* [com.moowork.node](https://plugins.gradle.org/plugin/com.moowork.node)
* [com.moowork.grunt](https://plugins.gradle.org/plugin/com.moowork.grunt)
* [com.moowork.gulp](https://plugins.gradle.org/plugin/com.moowork.gulp)
    * 利用例: [borysn/spring-boot-angular2](https://github.com/borysn/spring-boot-angular2)

### Git Plugin

build時にGitのハッシュを取得する

* Spring Boot Actuator向け
    * [com.gorylenko.gradle-git-properties](https://plugins.gradle.org/plugin/com.gorylenko.gradle-git-properties)
    * 参考: [SpringBootでリリースモジュールのGit情報を表示する](http://qiita.com/AkihiroTakamura/items/9f6a04de2a09e807efd8)


### Git Plugin

build時にGitのハッシュを取得する

* その他
    * [org.ajoberstar.grgit](https://plugins.gradle.org/plugin/org.ajoberstar.grgit)
    * GitのHEADを取得して、applicationの成果物にファイルとして含める。
    * 参考: [Gradle Goodness: Use Git Commit Id in Build Script](http://mrhaki.blogspot.jp/2015/04/gradle-goodness-use-git-commit-id-in.html)
    * 参考: [50.1.1. The distribution](https://docs.gradle.org/current/userguide/application_plugin.html#N15EBE)


### Dockerに関するPlugin

Dockerfileやdocker-compose.ymlを生成したり、dockerのコマンドを[docker-java](https://github.com/docker-java/docker-java)で実行する

* [com.bmuschko.docker-java-application](https://plugins.gradle.org/plugin/com.bmuschko.docker-java-application)
* [com.chrisgahlert.gradle-dcompose-plugin](https://plugins.gradle.org/plugin/com.chrisgahlert.gradle-dcompose-plugin)
* [com.palantir.docker](https://plugins.gradle.org/plugin/com.palantir.docker)
* [de.gesellix.docker](https://plugins.gradle.org/plugin/de.gesellix.docker)
    * 参考: [gesellix/gradle-docker-plugin-example](https://github.com/gesellix/gradle-docker-plugin-example)


### JRuby Gradle Plugin

GradleからJRubyを実行したい

* [com.github.jruby-gradle.base](https://plugins.gradle.org/plugin/com.github.jruby-gradle.base)

[Asciidoctor](http://asciidoctor.org/)を使いたい場合も

* [org.asciidoctor.gradle.asciidoctor](https://plugins.gradle.org/plugin/org.asciidoctor.gradle.asciidoctor)
    * 参考: [asciidoctor/asciidoctor-gradle-examples](https://github.com/asciidoctor/asciidoctor-gradle-examples)

### その他のPlugin

DBのマイグレーション

* [org.flywaydb.flyway](https://plugins.gradle.org/plugin/org.flywaydb.flyway)

AWSに関する操作

* [jp.classmethod.aws](https://plugins.gradle.org/plugin/jp.classmethod.aws)


### その他のPlugin

Nebula Plugin

* [Nebula: A collection of Gradle plugins, built by Netflix](http://nebula-plugins.github.io/)
* Netflixで利用されている、ビルドに関係するGradle Pluginのセット。


## まとめ

* Gradleは怖くない。
* 小さいタスクを作る。
* タスクを繋げる。
* 大きくなったら分割する。
* 困ったらPluginを探してみる。


## Let's enjoy Gradle!